# N Burning XGBoost FAQs Answered Neatly For Beginners
## Gradient boosted answers
![](images/unsplash.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@haithemfrd_off?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Haithem Ferdi</a>
        on 
        <a href='https://unsplash.com/s/photos/boost?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash.</a> All images are by the author unless specified otherwise.
    </strong>
</figcaption>

## Setup

In [10]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from matplotlib import rcParams
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split

warnings.filterwarnings("ignore")
rcParams["font.size"] = 15

In [11]:
iris = sns.load_dataset("iris").dropna()
penguins = sns.load_dataset("penguins").dropna()

In [12]:
i_train, i_target = iris.drop("species", axis=1), iris[["species"]]
p_train, p_target = penguins.drop("body_mass_g", axis=1), penguins[["body_mass_g"]]